<a href="https://colab.research.google.com/github/sbh69840/SOT/blob/main/Object_tracking_opencv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sbh69840/SOT-dataset.git

Cloning into 'SOT-dataset'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 17 (delta 2), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (17/17), done.


In [2]:
track_video = "track1"

In [4]:
import os
os.system("unzip SOT-dataset/{}.zip".format(track_video))

0

In [ ]:
# Resource : https://www.learnopencv.com/object-tracking-using-opencv-cpp-python/

import cv2
import sys

(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')￼

# Set up tracker.
# Instead of MIL, you can also use

tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
tracker_type = tracker_types[2]

if int(minor_ver) < 3:
    tracker = cv2.Tracker_create(tracker_type)
else:
    if tracker_type == 'BOOSTING':
        tracker = cv2.TrackerBoosting_create()
    if tracker_type == 'MIL':
        tracker = cv2.TrackerMIL_create()
    if tracker_type == 'KCF':
        tracker = cv2.TrackerKCF_create()
    if tracker_type == 'TLD':
        tracker = cv2.TrackerTLD_create()
    if tracker_type == 'MEDIANFLOW':
        tracker = cv2.TrackerMedianFlow_create()
    if tracker_type == 'GOTURN':
        tracker = cv2.TrackerGOTURN_create()
    if tracker_type == 'MOSSE':
        tracker = cv2.TrackerMOSSE_create()
    if tracker_type == "CSRT":
        tracker = cv2.TrackerCSRT_create()

In [5]:
frame_name = "1.jpg"
frame_name = frame_name.rjust(12,"0")
os.system("cp {0}/{1} /usr/local/share/jupyter/nbextensions/{1}".format(track_video,frame_name))

0

In [ ]:
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
from PIL import Image

html_canvas = """
<canvas id="canvas"></canvas>
<button id="capture">Capture</button>
<script>
var canvas = document.getElementById('canvas');
var width = %d;
var height = %d;
canvas.style.width = width+'px';
canvas.style.height = height+'px';
var ctx = canvas.getContext('2d');
ctx.canvas.width = width;
ctx.canvas.height = height;
var base_image = new Image();
base_image.src = '%s';
base_image.width = width+'px';
base_image.height = height+'px';
base_image.onload = function(){
  ctx.drawImage(base_image,0,0,width,height);
}
initDraw(canvas,ctx);
var x,y,width,height;
function initDraw(canvas,ctx) {
    function setMousePosition(e) {
        var ev = e || window.event; //Moz || IE
        if (ev.pageX) { //Moz
            mouse.x = ev.pageX + window.pageXOffset;
            mouse.y = ev.pageY + window.pageYOffset;
        } else if (ev.clientX) { //IE
            mouse.x = ev.clientX + document.body.scrollLeft;
            mouse.y = ev.clientY + document.body.scrollTop;
        }
    };

    var mouse = {
        x: 0,
        y: 0,
        startX: 0,
        startY: 0
    };
    var left=null,top=null,width=0,height=0;
    canvas.onmousemove = function (e) {
        setMousePosition(e);
        if (left !== null) {
            ctx.clearRect(0,0,canvas.width,canvas.height);
            ctx.drawImage(base_image,0,0,canvas.width,canvas.height);
            width = Math.abs(mouse.x - mouse.startX);
            height = Math.abs(mouse.y - mouse.startY);
            left = (mouse.x - mouse.startX < 0) ? mouse.x : mouse.startX;
            top = (mouse.y - mouse.startY < 0) ? mouse.y : mouse.startY;
            ctx.beginPath();
            ctx.lineWidth = "%d";
            ctx.strokeStyle = "%s";
            ctx.strokeRect(left, top, width, height);
            console.log(left,top,width,height);
            
            ctx.stroke();
        }
    }

    canvas.onclick = function (e) {
        if (left !== null) {
            window.x = left;
            window.y = top;
            window.width = width;
            window.height = height;
            left = null;
            canvas.style.cursor = "default";
            console.log("finsihed.");
        } else {
            console.log("begun.");
            mouse.startX = mouse.x;
            mouse.startY = mouse.y;
            left = mouse.x;
            top = mouse.y;
            canvas.style.cursor = "crosshair";
        }
    }
    
    
}
var button = document.getElementById('capture');
var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve([x,y,width,height]);
  }
});
</script>
"""

def selectROI(filename='1.jpg', w=400, h=200, line_width=3,color="yellow"):
  img = Image.open('/usr/local/share/jupyter/nbextensions/{}'.format(filename))
  display(HTML(html_canvas%(img.size[0],img.size[1],'/nbextensions/{}'.format(filename),line_width,color)))
  data = eval_js("data")
  return tuple(data)

In [ ]:
# Read video
    video = cv2.VideoCapture("videos/chaplin.mp4")

    # Exit if video not opened.
    if not video.isOpened():
        print "Could not open video"
        sys.exit()

    # Read first frame.
    ok, frame = video.read()
    if not ok:
        print 'Cannot read video file'
        sys.exit()
    
    # Define an initial bounding box
    bbox = (287, 23, 86, 320)

    # Uncomment the line below to select a different bounding box
    bbox = cv2.selectROI(frame, False)

    # Initialize tracker with first frame and bounding box
    ok = tracker.init(frame, bbox)

    while True:
        # Read a new frame
        ok, frame = video.read()
        if not ok:
            break
        
        # Start timer
        timer = cv2.getTickCount()

        # Update tracker
        ok, bbox = tracker.update(frame)

        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);

        # Draw bounding box
        if ok:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
        else :
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

        # Display tracker type on frame
        cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);
    
        # Display FPS on frame
        cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);

        # Display result
        cv2.imshow("Tracking", frame)

        # Exit if ESC pressed
        k = cv2.waitKey(1) & 0xff
        if k == 27 : break
